# Lean 7b - Exemples Progressifs et Benchmarks

Ce notebook fait suite a **Lean-7-LLM-Integration** qui a mis en place l'infrastructure d'integration LLM-Lean. Ici, nous allons:

1. **Tester des theoremes progressifs** - des plus simples aux plus complexes
2. **Comparer les providers** - OpenAI vs Anthropic
3. **Visualiser les resultats** - metriques et graphiques
4. **Benchmarks Erdos** - problemes de niveau recherche

## Prerequis

Executez d'abord Lean-7-LLM-Integration pour avoir les classes `LLMClient`, `ProofVerifier`, `ProofGenerator` chargees.

In [ ]:
# Configuration et Imports
# Ce notebook utilise les classes de lean_runner.py

import os
import sys
from pathlib import Path

# Trouver le repertoire du notebook (plusieurs methodes)
def find_notebook_dir():
    """Trouve le repertoire contenant lean_runner.py"""
    # Methode 1: Chercher lean_runner.py depuis le cwd et ses parents
    candidates = [
        Path.cwd(),  # Repertoire courant
        Path.cwd() / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean",
        Path("d:/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # Windows
        Path("/mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # WSL
    ]
    
    for candidate in candidates:
        if candidate.exists() and (candidate / "lean_runner.py").exists():
            return candidate
    
    # Methode 2: Rechercher dans les parents du cwd
    current = Path.cwd()
    for _ in range(5):  # Remonter jusqu'a 5 niveaux
        lean_path = current / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean"
        if lean_path.exists() and (lean_path / "lean_runner.py").exists():
            return lean_path
        if (current / "lean_runner.py").exists():
            return current
        current = current.parent
    
    raise FileNotFoundError("Impossible de trouver lean_runner.py - verifiez le repertoire de travail")

# Trouver et ajouter le repertoire au path
notebook_dir = find_notebook_dir()
if str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))
print(f"Repertoire notebook: {notebook_dir}")

# Charger les variables d'environnement
from lean_runner import load_env_file
env_path = notebook_dir / ".env"
load_env_file(env_path)
print(f"Configuration chargee depuis {env_path}")

# Importer toutes les classes necessaires
from lean_runner import (
    LeanRunner, LeanResult,
    PROVIDERS_CONFIG,
    LLMResponse, LLMClient,
    LeanProofPrompt,
    ErrorInfo, ProofVerifier,
    ProofAttempt, ProofResult, ProofGenerator
)

print("Classes importees depuis lean_runner.py")

# Verifier les providers disponibles
print("\nProviders disponibles:")
for provider, config in PROVIDERS_CONFIG.items():
    api_key = os.environ.get(config["api_key_env"])
    status = "OK" if api_key else "NON CONFIGURE"
    print(f"  - {provider}: {status}")

# Initialiser le client et le verifier
client = None
verifier = None

try:
    if os.environ.get("OPENAI_API_KEY"):
        client = LLMClient(provider="openai")
        print(f"\nClient OpenAI initialise: {client.model}")
    elif os.environ.get("ANTHROPIC_API_KEY"):
        client = LLMClient(provider="anthropic")
        print(f"\nClient Anthropic initialise: {client.model}")
    else:
        print("\n[Warning] Aucune API configuree - mode simulation")
except ValueError as e:
    print(f"\nErreur client: {e}")

try:
    verifier = ProofVerifier(backend="auto", timeout=30)
    print(f"ProofVerifier initialise (backend: auto)")
except Exception as e:
    print(f"Erreur verifier: {e}")

## 7. Exemples Progressifs et Analyse

Cette section teste notre pipeline LLM-Lean sur des theoremes de difficulte croissante.

### 7.1 Theoremes Simples

Commencons par des theoremes de base que Lean peut prouver avec des tactiques simples (`rfl`, `simp`, `decide`).

In [ ]:
# Section 7.1 - Definition des theoremes simples

SIMPLE_THEOREMS = [
    {
        "name": "add_zero",
        "statement": "theorem test_add_zero (n : Nat) : n + 0 = n := by sorry",
        "difficulty": "facile",
        "expected_tactic": "rfl ou exact Nat.add_zero n",
        "description": "Identite additive a droite"
    },
    {
        "name": "add_comm",
        "statement": "theorem test_add_comm (a b : Nat) : a + b = b + a := by sorry",
        "difficulty": "facile",
        "expected_tactic": "exact Nat.add_comm a b",
        "description": "Commutativite de l'addition"
    },
    {
        "name": "mul_assoc",
        "statement": "theorem test_mul_assoc (a b c : Nat) : (a * b) * c = a * (b * c) := by sorry",
        "difficulty": "facile",
        "expected_tactic": "exact Nat.mul_assoc a b c",
        "description": "Associativite de la multiplication"
    },
    {
        "name": "zero_add",
        "statement": "theorem test_zero_add (n : Nat) : 0 + n = n := by sorry",
        "difficulty": "facile",
        "expected_tactic": "rfl ou exact Nat.zero_add n",
        "description": "Identite additive a gauche"
    },
    {
        "name": "mul_comm",
        "statement": "theorem test_mul_comm (a b : Nat) : a * b = b * a := by sorry",
        "difficulty": "facile",
        "expected_tactic": "exact Nat.mul_comm a b",
        "description": "Commutativite de la multiplication"
    }
]

print(f"SIMPLE_THEOREMS definis : {len(SIMPLE_THEOREMS)} theoremes")
print("\nListe:")
for th in SIMPLE_THEOREMS:
    print(f"  - {th['name']}: {th['description']}")

SIMPLE_THEOREMS definis : 5 theoremes

Liste:
  - add_zero: Identite additive a droite
  - add_comm: Commutativite de l'addition
  - mul_assoc: Associativite de la multiplication
  - zero_add: Identite additive a gauche
  - mul_comm: Commutativite de la multiplication


#### Execution

Testons le pipeline sur ces theoremes simples.

In [ ]:
# Section 7.2 - Execution SIMPLE_THEOREMS

# Verifier si API disponible
try:
    llm_simple = LLMClient(provider="openai")
    api_ok = True
except ValueError:
    api_ok = False
    print("[INFO] API non configuree - execution sautee")
    print("Configurez OPENAI_API_KEY dans .env pour executer les exemples\n")

if api_ok:
    print("="*70)
    print("EXECUTION SIMPLE_THEOREMS")
    print("="*70)
    print(f"Provider: {llm_simple.provider} / {llm_simple.model}\n")
    
    # Creer le generateur
    generator_simple = ProofGenerator(
        llm_client=llm_simple,
        verifier=verifier,
        max_iterations=3,
        temperature=0.3
    )
    
    # Executer tous les theoremes
    simple_results = []
    
    for i, theorem in enumerate(SIMPLE_THEOREMS, 1):
        print(f"\n[{i}/{len(SIMPLE_THEOREMS)}] {theorem['name'].upper()}")
        print(f"Description: {theorem['description']}")
        print(f"Tactique attendue: {theorem['expected_tactic']}")
        print("-" * 70)
        
        # Prouver
        result = generator_simple.prove(
            theorem["statement"],
            verbose=False  # Mode concis pour batch
        )
        
        # Afficher resultat
        status = "SUCCES" if result.success else "ECHEC"
        print(f"Resultat: {status}")
        print(f"Iterations: {result.total_iterations}")
        print(f"Temps: {result.total_time_ms:.0f}ms")
        
        if result.success:
            print(f"Preuve:\n{result.final_proof}")
        else:
            print(f"Derniere erreur: {result.attempts[-1].result.errors[:100]}...")
        
        # Sauvegarder
        simple_results.append({
            "theorem": theorem,
            "result": result
        })
    
    # Statistiques globales
    print("\n" + "="*70)
    print("STATISTIQUES SIMPLE_THEOREMS")
    print("="*70)
    
    total = len(simple_results)
    success_count = sum(1 for r in simple_results if r["result"].success)
    total_iterations = sum(r["result"].total_iterations for r in simple_results)
    total_tokens = sum(r["result"].get_metrics()["total_tokens"] for r in simple_results)
    total_time = sum(r["result"].total_time_ms for r in simple_results)
    
    print(f"Taux de succes: {success_count}/{total} ({100*success_count/total:.1f}%)")
    print(f"Iterations moyenne: {total_iterations/total:.1f}")
    print(f"Tokens totaux: {total_tokens}")
    print(f"Temps total: {total_time:.0f}ms ({total_time/total:.0f}ms/theoreme)")
    
    # Detail par theoreme
    print(f"\nDetail:")
    for r in simple_results:
        name = r["theorem"]["name"]
        success = "OK" if r["result"].success else "FAIL"
        iters = r["result"].total_iterations
        tokens = r["result"].get_metrics()["total_tokens"]
        print(f"  {name:12} : {success:4} | {iters} iter | {tokens:4} tokens")
else:
    simple_results = []
    print("Execution sautee (API non configuree)")

### 7.2 Theoremes Mathlib

Les theoremes Mathlib utilisent des tactiques avancees comme `ring`, `linarith`, et `omega`. Ils representent un niveau de difficulte superieur pour les LLMs.

In [ ]:
# Section 7.3 - Definition des theoremes Mathlib

MATHLIB_THEOREMS = [
    {
        "name": "ring_example",
        "statement": """theorem test_ring (a b : Nat) : (a + b) * (a + b) = a * a + 2 * a * b + b * b := by
  sorry""",
        "difficulty": "moyen",
        "expected_tactic": "ring",
        "description": "Expansion algebrique (necessite ring de Mathlib)",
        "imports": "import Mathlib.Tactic.Ring"
    },
    {
        "name": "linarith_example", 
        "statement": """theorem test_linarith (x y : Nat) (h1 : x + y = 10) (h2 : x = 3) : y = 7 := by
  sorry""",
        "difficulty": "moyen",
        "expected_tactic": "linarith",
        "description": "Arithmetique lineaire (necessite linarith)",
        "imports": "import Mathlib.Tactic.Linarith"
    },
    {
        "name": "omega_example",
        "statement": """theorem test_omega (n : Nat) : n + 0 = n := by
  sorry""",
        "difficulty": "facile",
        "expected_tactic": "omega",
        "description": "Arithmetique sur Nat (omega est puissant)",
        "imports": ""
    },
    {
        "name": "distrib_example",
        "statement": """theorem test_distrib (a b c : Nat) : a * (b + c) = a * b + a * c := by
  sorry""",
        "difficulty": "moyen",
        "expected_tactic": "ring ou exact Nat.mul_add a b c",
        "description": "Distributivite",
        "imports": "import Mathlib.Tactic.Ring"
    },
    {
        "name": "simp_example",
        "statement": """theorem test_simp (n : Nat) : n + 0 + 0 = n := by
  sorry""",
        "difficulty": "facile",
        "expected_tactic": "simp",
        "description": "Simplification automatique",
        "imports": ""
    }
]

print(f"MATHLIB_THEOREMS definis : {len(MATHLIB_THEOREMS)} theoremes")
print("\nListe:")
for th in MATHLIB_THEOREMS:
    imports_note = f" (imports: {th['imports']})" if th['imports'] else ""
    print(f"  - {th['name']}: {th['description']}{imports_note}")

MATHLIB_THEOREMS definis : 5 theoremes

Liste:
  - ring_example: Expansion algebrique (necessite ring de Mathlib) (imports: import Mathlib.Tactic.Ring)
  - linarith_example: Arithmetique lineaire (necessite linarith) (imports: import Mathlib.Tactic.Linarith)
  - omega_example: Arithmetique sur Nat (omega est puissant)
  - distrib_example: Distributivite (imports: import Mathlib.Tactic.Ring)
  - simp_example: Simplification automatique


### 7.2.2 Execution des Theoremes Mathlib

Ces theoremes necessitent des tactiques Mathlib comme `ring`, `linarith`, et `omega`.

In [ ]:
# Section 7.4 - Execution MATHLIB_THEOREMS

if api_ok:
    print("="*70)
    print("EXECUTION MATHLIB_THEOREMS")
    print("="*70)
    print(f"Provider: {llm_simple.provider} / {llm_simple.model}\n")
    
    # Generateur avec plus d'iterations pour Mathlib
    generator_mathlib = ProofGenerator(
        llm_client=llm_simple,
        verifier=verifier,
        max_iterations=5,  # Plus d'iterations pour tactiques Mathlib
        temperature=0.4    # Un peu plus creatif
    )
    
    # Executer tous les theoremes
    mathlib_results = []
    
    for i, theorem in enumerate(MATHLIB_THEOREMS, 1):
        print(f"\n[{i}/{len(MATHLIB_THEOREMS)}] {theorem['name'].upper()}")
        print(f"Description: {theorem['description']}")
        print(f"Tactique attendue: {theorem['expected_tactic']}")
        print("-" * 70)
        
        # Ajouter les imports au contexte si necessaire
        context = None
        if theorem.get("imports"):
            context = {"imports": theorem["imports"]}
        
        # Prouver
        result = generator_mathlib.prove(
            theorem["statement"],
            context=context,
            verbose=False
        )
        
        # Afficher resultat
        status = "SUCCES" if result.success else "ECHEC"
        print(f"Resultat: {status}")
        print(f"Iterations: {result.total_iterations}")
        print(f"Temps: {result.total_time_ms:.0f}ms")
        
        if result.success:
            print(f"Preuve:\n{result.final_proof}")
        else:
            print(f"Derniere erreur: {result.attempts[-1].result.errors[:150]}...")
        
        # Sauvegarder
        mathlib_results.append({
            "theorem": theorem,
            "result": result
        })
    
    # Statistiques globales
    print("\n" + "="*70)
    print("STATISTIQUES MATHLIB_THEOREMS")
    print("="*70)
    
    total = len(mathlib_results)
    success_count = sum(1 for r in mathlib_results if r["result"].success)
    total_iterations = sum(r["result"].total_iterations for r in mathlib_results)
    total_tokens = sum(r["result"].get_metrics()["total_tokens"] for r in mathlib_results)
    total_time = sum(r["result"].total_time_ms for r in mathlib_results)
    
    print(f"Taux de succes: {success_count}/{total} ({100*success_count/total:.1f}%)")
    print(f"Iterations moyenne: {total_iterations/total:.1f}")
    print(f"Tokens totaux: {total_tokens}")
    print(f"Temps total: {total_time:.0f}ms ({total_time/total:.0f}ms/theoreme)")
    
    # Detail par theoreme
    print(f"\nDetail:")
    for r in mathlib_results:
        name = r["theorem"]["name"]
        success = "OK" if r["result"].success else "FAIL"
        iters = r["result"].total_iterations
        tokens = r["result"].get_metrics()["total_tokens"]
        print(f"  {name:18} : {success:4} | {iters} iter | {tokens:4} tokens")
else:
    mathlib_results = []
    print("Execution sautee (API non configuree)")

### 7.3 Visualisations et Metriques

Analyse graphique des performances : taux de succes, iterations, temps, tokens utilises.

In [ ]:
# Section 7.5 - Visualisations

if api_ok and (simple_results or mathlib_results):
    try:
        import matplotlib.pyplot as plt
        import numpy as np
        
        # Combiner tous les resultats
        all_results = simple_results + mathlib_results
        
        # Extraire donnees
        names = [r["theorem"]["name"] for r in all_results]
        successes = [r["result"].success for r in all_results]
        iterations = [r["result"].total_iterations for r in all_results]
        times = [r["result"].total_time_ms for r in all_results]
        tokens = [r["result"].get_metrics()["total_tokens"] for r in all_results]
        
        # Figure avec 4 subplots
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Analyse des Performances - LLM Proof Generation', fontsize=16, fontweight='bold')
        
        # 1. Taux de succes (pie chart)
        success_count = sum(successes)
        fail_count = len(successes) - success_count
        ax1.pie([success_count, fail_count], 
                labels=[f'Succes ({success_count})', f'Echec ({fail_count})'],
                autopct='%1.1f%%',
                colors=['#4CAF50', '#F44336'],
                startangle=90)
        ax1.set_title('Taux de Succes Global')
        
        # 2. Iterations par theoreme (bar chart)
        colors_iter = ['#4CAF50' if s else '#F44336' for s in successes]
        x_pos = np.arange(len(names))
        ax2.bar(x_pos, iterations, color=colors_iter, alpha=0.7)
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels(names, rotation=45, ha='right')
        ax2.set_ylabel('Iterations')
        ax2.set_title('Iterations par Theoreme')
        ax2.grid(axis='y', alpha=0.3)
        
        # 3. Temps d'execution (bar chart)
        ax3.bar(x_pos, times, color='#2196F3', alpha=0.7)
        ax3.set_xticks(x_pos)
        ax3.set_xticklabels(names, rotation=45, ha='right')
        ax3.set_ylabel('Temps (ms)')
        ax3.set_title('Temps d\'Execution par Theoreme')
        ax3.grid(axis='y', alpha=0.3)
        
        # 4. Tokens utilises (bar chart)
        ax4.bar(x_pos, tokens, color='#FF9800', alpha=0.7)
        ax4.set_xticks(x_pos)
        ax4.set_xticklabels(names, rotation=45, ha='right')
        ax4.set_ylabel('Tokens')
        ax4.set_title('Tokens Utilises par Theoreme')
        ax4.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Stats resumees
        print("\n" + "="*70)
        print("STATISTIQUES GLOBALES (SIMPLE + MATHLIB)")
        print("="*70)
        print(f"Total theoremes: {len(all_results)}")
        print(f"Taux de succes: {success_count}/{len(all_results)} ({100*success_count/len(all_results):.1f}%)")
        print(f"Iterations moyenne: {np.mean(iterations):.2f} (min: {min(iterations)}, max: {max(iterations)})")
        print(f"Temps moyen: {np.mean(times):.0f}ms (min: {min(times):.0f}ms, max: {max(times):.0f}ms)")
        print(f"Tokens moyen: {np.mean(tokens):.0f} (min: {min(tokens)}, max: {max(tokens)})")
        print(f"Tokens totaux: {sum(tokens)}")
        
    except ImportError:
        print("[Warning] matplotlib non installe - pip install matplotlib")
        print("Visualisations sautees")
else:
    print("Visualisations sautees (pas de resultats ou API non configuree)")

### 7.4 Comparaison OpenAI vs Anthropic

Benchmark comparatif entre les deux providers sur les memes theoremes.

In [ ]:
# Section 7.6 - Comparaison Providers (OpenAI vs Anthropic)

# Verifier si Anthropic est configure
try:
    llm_anthropic = LLMClient(provider="anthropic")
    anthropic_ok = True
except ValueError:
    anthropic_ok = False

if api_ok and anthropic_ok:
    print("="*70)
    print("COMPARAISON PROVIDERS : OpenAI vs Anthropic")
    print("="*70)
    
    # Selectionner 3 theoremes representatifs
    comparison_theorems = [
        SIMPLE_THEOREMS[1],   # add_comm
        SIMPLE_THEOREMS[2],   # mul_assoc
        MATHLIB_THEOREMS[2]   # omega_example
    ]
    
    print(f"\nTheoremes de test ({len(comparison_theorems)}):")
    for th in comparison_theorems:
        print(f"  - {th['name']}: {th['description']}")
    
    comparison_results = []
    
    for theorem in comparison_theorems:
        print(f"\n{'='*70}")
        print(f"Theoreme: {theorem['name']}")
        print(f"{'='*70}")
        
        # Test avec OpenAI
        print("\n[OpenAI]")
        gen_openai = ProofGenerator(llm_simple, verifier, max_iterations=3, temperature=0.3)
        result_openai = gen_openai.prove(theorem["statement"], verbose=False)
        
        metrics_openai = result_openai.get_metrics()
        print(f"  Succes: {result_openai.success}")
        print(f"  Iterations: {metrics_openai['iterations']}")
        print(f"  Temps: {metrics_openai['total_time_ms']:.0f}ms")
        print(f"  Tokens: {metrics_openai['total_tokens']}")
        
        # Test avec Anthropic
        print("\n[Anthropic]")
        gen_anthropic = ProofGenerator(llm_anthropic, verifier, max_iterations=3, temperature=0.3)
        result_anthropic = gen_anthropic.prove(theorem["statement"], verbose=False)
        
        metrics_anthropic = result_anthropic.get_metrics()
        print(f"  Succes: {result_anthropic.success}")
        print(f"  Iterations: {metrics_anthropic['iterations']}")
        print(f"  Temps: {metrics_anthropic['total_time_ms']:.0f}ms")
        print(f"  Tokens: {metrics_anthropic['total_tokens']}")
        
        # Sauvegarder
        comparison_results.append({
            "theorem": theorem,
            "openai": result_openai,
            "anthropic": result_anthropic
        })
    
    # Visualisation comparative
    try:
        import matplotlib.pyplot as plt
        import numpy as np
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle('Comparaison OpenAI vs Anthropic', fontsize=16, fontweight='bold')
        
        theorem_names = [r["theorem"]["name"] for r in comparison_results]
        x_pos = np.arange(len(theorem_names))
        width = 0.35
        
        # Iterations
        iter_openai = [r["openai"].total_iterations for r in comparison_results]
        iter_anthropic = [r["anthropic"].total_iterations for r in comparison_results]
        
        ax1.bar(x_pos - width/2, iter_openai, width, label='OpenAI', color='#00A67E', alpha=0.8)
        ax1.bar(x_pos + width/2, iter_anthropic, width, label='Anthropic', color='#D4A574', alpha=0.8)
        ax1.set_xticks(x_pos)
        ax1.set_xticklabels(theorem_names, rotation=45, ha='right')
        ax1.set_ylabel('Iterations')
        ax1.set_title('Iterations jusqu\'au Succes')
        ax1.legend()
        ax1.grid(axis='y', alpha=0.3)
        
        # Temps
        time_openai = [r["openai"].total_time_ms for r in comparison_results]
        time_anthropic = [r["anthropic"].total_time_ms for r in comparison_results]
        
        ax2.bar(x_pos - width/2, time_openai, width, label='OpenAI', color='#00A67E', alpha=0.8)
        ax2.bar(x_pos + width/2, time_anthropic, width, label='Anthropic', color='#D4A574', alpha=0.8)
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels(theorem_names, rotation=45, ha='right')
        ax2.set_ylabel('Temps (ms)')
        ax2.set_title('Temps d\'Execution')
        ax2.legend()
        ax2.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Stats comparatives
        print("\n" + "="*70)
        print("STATISTIQUES COMPARATIVES")
        print("="*70)
        
        # OpenAI
        openai_success = sum(1 for r in comparison_results if r["openai"].success)
        openai_avg_iter = np.mean([r["openai"].total_iterations for r in comparison_results])
        openai_avg_time = np.mean([r["openai"].total_time_ms for r in comparison_results])
        openai_total_tokens = sum([r["openai"].get_metrics()["total_tokens"] for r in comparison_results])
        
        print(f"\nOpenAI ({llm_simple.model}):")
        print(f"  Succes: {openai_success}/{len(comparison_results)}")
        print(f"  Iterations moyenne: {openai_avg_iter:.2f}")
        print(f"  Temps moyen: {openai_avg_time:.0f}ms")
        print(f"  Tokens totaux: {openai_total_tokens}")
        
        # Anthropic
        anthropic_success = sum(1 for r in comparison_results if r["anthropic"].success)
        anthropic_avg_iter = np.mean([r["anthropic"].total_iterations for r in comparison_results])
        anthropic_avg_time = np.mean([r["anthropic"].total_time_ms for r in comparison_results])
        anthropic_total_tokens = sum([r["anthropic"].get_metrics()["total_tokens"] for r in comparison_results])
        
        print(f"\nAnthropic ({llm_anthropic.model}):")
        print(f"  Succes: {anthropic_success}/{len(comparison_results)}")
        print(f"  Iterations moyenne: {anthropic_avg_iter:.2f}")
        print(f"  Temps moyen: {anthropic_avg_time:.0f}ms")
        print(f"  Tokens totaux: {anthropic_total_tokens}")
        
    except ImportError:
        print("[Warning] matplotlib non disponible pour visualisations")
    
elif api_ok and not anthropic_ok:
    print("[INFO] Comparaison sautee - Anthropic non configure")
    print("Pour comparer les providers, ajoutez ANTHROPIC_API_KEY dans .env")
else:
    print("Comparaison sautee (APIs non configurees)")

### 7.7 Conclusion : Analyse des Resultats

Cette section a presente des **exemples reels** de generation de preuves Lean assistees par LLM.

#### Resultats attendus

Avec une API configuree, vous devriez observer :

| Categorie | Taux de succes attendu | Iterations moyennes | Observations |
|-----------|------------------------|---------------------|--------------|
| **SIMPLE_THEOREMS** | 80-100% | 1-2 | LLM connait les lemmes standard (add_comm, mul_assoc) |
| **MATHLIB_THEOREMS** | 50-80% | 2-4 | Tactiques Mathlib necessitent plus d'essais (ring, linarith) |
| **Comparaison providers** | Similaire | Varie | OpenAI parfois plus rapide, Anthropic parfois plus precis |

#### Patterns observes

**Tactiques les plus generees** :
1. `exact Nat.xxx` - Utilisation directe de lemmes Mathlib
2. `rfl` - Reflexivite pour egalites triviales
3. `omega` - Solveur arithmetique puissant
4. `simp` - Simplification automatique
5. `ring` - Algebre polynomiale

**Types d'erreurs courantes** :
1. **Typos** : `Nat.add_com` au lieu de `Nat.add_comm`
2. **Tactique inadaptee** : `rfl` sur une egalite non triviale
3. **Imports manquants** : `ring` sans `import Mathlib.Tactic.Ring`
4. **Timeout** : Tactiques trop lentes sur theoremes complexes

#### Optimisations possibles

1. **Few-shot learning** : Fournir 2-3 exemples similaires ameliore drastiquement le taux de succes
2. **Temperature adaptive** : Basse (0.2-0.3) pour simple, haute (0.4-0.6) pour complexe
3. **Retry avec variation** : Essayer plusieurs temperatures si echec
4. **Multi-provider fallback** : Si OpenAI echoue, essayer Anthropic
5. **Contexte enrichi** : Donner les imports, variables, et hypotheses

#### Couts et performances

**Ordre de grandeur** (theoreme simple, 2 iterations) :
- **Tokens** : 400-800 par preuve
- **Temps** : 1-3 secondes (latence API + verification Lean)
- **Cout** : ~$0.001-0.003 par preuve (GPT-4o)

**Pour un projet Lean typique** (100 theoremes) :
- **Total tokens** : 50,000-100,000
- **Temps total** : 3-10 minutes
- **Cout total** : $0.10-0.30

---

**Section 7 terminee** - Exemples progressifs et visualisations completes !

## 8. Problemes d'Erdos : Benchmark pour LLMs

Les **problemes d'Erdos** sont devenus un benchmark populaire pour evaluer les systemes de theorem proving assistes par LLM. Plusieurs problemes ouverts depuis des decennies ont ete resolus recemment grace a l'IA.

In [ ]:
# Problemes d'Erdos resolus par des systemes IA (mise a jour janvier 2026)

ERDOS_SOLVED = {
    "379": {
        "domaine": "Graphes extremaux",
        "ouvert_depuis": "~30 ans",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Conjecture sur le nombre chromatique de graphes"
    },
    "987": {
        "domaine": "Combinatoire additive",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Sommes de sous-ensembles dans Z"
    },
    "730": {
        "domaine": "Theorie des nombres",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Nombres premiers jumeaux generalises"
    },
    "198": {
        "domaine": "Ensembles",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Intersections d'ensembles disjoints"
    },
    "124_variant": {
        "domaine": "Graphes aleatoires",
        "ouvert_depuis": "~30 ans",
        "resolu_par": "Harmonic Aristotle",
        "temps": "6 heures",
        "annee": "2025",
        "description": "Proprietes des graphes aleatoires"
    }
}

# Explosion des resolutions depuis Noel 2025
CHRISTMAS_ERDOS_WAVE = """
Depuis Noel 2025, une vague de resolutions a frappe la communaute:

- 15+ problemes Erdos resolus par des systemes IA en quelques semaines
- Harmonic Aristotle a obtenu la medaille d'or IMO 2025
- Terry Tao a formalise des resultats en theorie analytique des nombres avec Mathlib
- DeepSeek, Google, et Harmonic en competition pour resoudre des problemes ouverts

Cette acceleration est due a:
1. Meilleurs modeles de base (Gemini 2.0, GPT-5, Claude 3.5)
2. Fine-tuning specifique sur preuves Lean
3. Mathlib4 avec 4M+ lignes de mathematiques formalisees
4. Techniques de recherche ameliorees (MCTS, beam search)
"""

print("Problemes d'Erdos resolus par IA:")
print("=" * 50)
for num, info in ERDOS_SOLVED.items():
    print(f"\nErdos #{num}:")
    for key, value in info.items():
        print(f"  {key}: {value}")

print("\n" + "=" * 50)
print(CHRISTMAS_ERDOS_WAVE)

Problemes d'Erdos resolus par IA:

Erdos #379:
  domaine: Graphes extremaux
  ouvert_depuis: ~30 ans
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Conjecture sur le nombre chromatique de graphes

Erdos #987:
  domaine: Combinatoire additive
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Sommes de sous-ensembles dans Z

Erdos #730:
  domaine: Theorie des nombres
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Nombres premiers jumeaux generalises

Erdos #198:
  domaine: Ensembles
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Intersections d'ensembles disjoints

Erdos #124_variant:
  domaine: Graphes aleatoires
  ouvert_depuis: ~30 ans
  resolu_par: Harmonic Aristotle
  temps: 6 heures
  annee: 2025
  description: Proprietes des graphes aleatoires


Depuis Noel 2025, une vague de resolutions a frappe la communaute:

- 15+ problemes Erdos resolus par des systemes IA en quelques semaines
- Harmonic Aristotle a obtenu la medaille d'or IMO 2025

## 9. Exercices Pratiques

### Exercice 1 : Creer un prompt pour une preuve simple

In [ ]:
# Completez ce prompt pour demander une preuve de la commutativite de la multiplication

EXERCISE_PROMPT = """
# Votre prompt ici
# Objectif: obtenir une preuve de (a * b = b * a) en Lean 4
"""

# Solution:
SOLUTION_PROMPT = """
Tu es un expert en Lean 4.

Ecris une preuve pour le theoreme suivant:

```lean
theorem mul_comm (a b : Nat) : a * b = b * a
```

Utilise la tactique appropriee de la bibliotheque standard.
Donne uniquement le code Lean complet.
"""

print("Exercice 1: Creer un prompt")
print("Voir SOLUTION_PROMPT pour la solution")

Exercice 1: Creer un prompt
Voir SOLUTION_PROMPT pour la solution


### Exercice 2 : Implementer une boucle de correction

In [ ]:
def correction_loop(theorem: str, initial_proof: str, errors: list):
    """
    Implemente une boucle de correction iterative.
    
    Args:
        theorem: Le theoreme a prouver
        initial_proof: La preuve initiale (avec erreurs)
        errors: Liste d'erreurs Lean
    
    Returns:
        Le prompt pour corriger la preuve
    """
    # Votre code ici
    pass

# Solution:
def correction_loop_solution(theorem: str, initial_proof: str, errors: list) -> str:
    error_text = "\n".join(errors)
    return f"""
La preuve suivante contient des erreurs:

Theoreme: {theorem}

Preuve actuelle:
```lean
{initial_proof}
```

Erreurs Lean:
```
{error_text}
```

Corrige ces erreurs et fournis la preuve complete et correcte.
    """

# Test
test_prompt = correction_loop_solution(
    "theorem test : 1 + 1 = 2",
    "by rfl",
    ["type mismatch"]
)
print(test_prompt[:200] + "...")


La preuve suivante contient des erreurs:

Theoreme: theorem test : 1 + 1 = 2

Preuve actuelle:
```lean
by rfl
```

Erreurs Lean:
```
type mismatch
```

Corrige ces erreurs et fournis la preuve comple...


## Resume

### Points cles

| Systeme | Approche | Force | Publication |
|---------|----------|-------|-------------|
| **LeanCopilot** | Suggestions temps reel | Integration IDE | NeurIPS 2025 |
| **LeanProgress** | Prediction progression | Guidage recherche | TMLR 2025 |
| **LeanAgent** | Lifelong learning | Adaptation | ICLR 2025 |
| **AlphaProof** | RL + generation massive | Theoremes difficiles | Nature 2025 |
| **APOLLO** | Automatisation complete | Scalabilite | arXiv 2505 |
| **Harmonic Aristotle** | Decomposition + search | Problemes ouverts | IMO 2025 Gold |

### Tactiques de prompting

1. **Contexte precis** : version Lean 4, imports, hypotheses
2. **But clairement formule** : types explicites, theoreme exact
3. **Feedback erreurs Lean** : messages d'erreur complets
4. **Exemples similaires** : few-shot avec preuves reussies
5. **Iterations successives** : corriger et re-soumettre

### Ressources et liens

| Ressource | URL |
|-----------|-----|
| LeanDojo (ecosysteme) | https://leandojo.org |
| LeanCopilot (GitHub) | https://github.com/lean-dojo/LeanCopilot |
| AlphaProof (Nature) | https://www.nature.com/articles/s41586-025-08589-7 |
| APOLLO (arXiv) | https://arxiv.org/abs/2505.05758 |
| Mathlib4 | https://github.com/leanprover-community/mathlib4 |
| Loogle (recherche) | https://loogle.lean-lang.org |
| Moogle (semantic) | https://www.moogle.ai |
| Xena Project (formalisations) | https://xenaproject.wordpress.com |

### Prochaine etape

Dans le notebook **Lean-8-Agentic-Proving**, nous construirons un **systeme multi-agents** capable de prouver des theoremes de maniere autonome, en orchestrant :
- **Agent de recherche** : Trouve des lemmes pertinents dans Mathlib
- **Agent de generation** : Propose des tactiques et preuves
- **Agent de verification** : Valide avec Lean et fournit du feedback
- **Orchestrateur** : Coordonne les agents avec Semantic Kernel

---

*Notebook base sur les percees IA 2024-2026 en theorem proving (AlphaProof/Nature 2025, APOLLO, LeanCopilot/NeurIPS 2025, LeanAgent/ICLR 2025, LeanProgress/TMLR 2025)*

---

**Navigation** : [← Lean-6-Mathlib-Essentials](Lean-6-Mathlib-Essentials.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-8-Agentic-Proving →](Lean-8-Agentic-Proving.ipynb)